In this notebook, we'll work through some basic features of Julia while building up some code to generate a single transverse field Ising model Hamiltonian. Our goal is to cover:
- Basic functions
- Methods vs functions
- Multiple dispatch
- Vectorization/broadcasting
- Structs
- Using packages
- Unit tests

In Julia we can call functions that are defined in "base Julia" itself:

In [ ]:
abs(-2)

In [ ]:
a = 10
b = 5.5
a * b

In [ ]:
A = rand(5, 5)
B = rand(5, 10)
A * B

In [ ]:
@which A * B

In [ ]:
C = rand(5)
D = rand(1, 5)
C * D

In [ ]:
@which C * D

In fact, we can ask Julia to show us *all* the type combinations for which a method of `*` is defined:

In [ ]:
methods(*)

We can define our own functions, but sometimes it's nice to be concise.

In [ ]:
function my_mult(A, B)
   return A*B
end
@assert my_mult(A, B) == A*B

In [ ]:
my_mult2(A, B) = A*B
@assert my_mult2(A, B) == A*B

One thing that makes Julia a good choice for scientific computing is that it is often very fast. But it's easy to write slow Julia code (it's easy to write bad code in any language). Let's examine the difference between a fast Julia function and a slow one, and see how we could spot the issue and fix it.

In [ ]:
function my_adder(A, B)
    C = 1
    C += A + B
    return C
end

Julia allows us to examine the final instructions emitted by LLVM after the compilation process.

In [ ]:
@code_llvm my_mult(1, 3)

In [ ]:
@code_llvm my_mult(1.1, 3.3)

Julia's compiler can generate highly efficient code that is correctly specialized on the input type, but **only** if the code is **type stable**. Let's see what that means:

In [ ]:
@code_warntype my_adder(1, 2)

In [ ]:
@code_llvm my_adder(1, 2)

In [ ]:
@code_warntype my_adder(1.2, 2.3)

In [ ]:
@code_llvm my_adder(1.2, 2.3)

In the case with floating point arguments, `my_adder` has extra instructions related to type promotion. How can we fix this and ensure type stability?

In [ ]:
function my_adder2(A::T, B::T) where {T}
    C = one(T)
    C += A + B
    return C
end

In [ ]:
@code_warntype my_adder2(1.2, 2.3)

In [ ]:
@code_warntype my_adder2(1, 2)

This seems to have helped. But what if `A` and `B` were of different types (e.g. were not both of type `T`)? We could write additional methods to *promote* them to a joint type (e.g. promote an `Int` to a `Float64` and then add them). `@code_warntype` and more sophisticated tools like [Cthulhu](https://github.com/JuliaDebug/Cthulhu.jl) can be very helpful ways to diagnose type instability and performance problems in your code.

Julia has a lot of sophisticated functionality for working with arrays, which makes it a great language for our purposes.

In [ ]:
M = zeros(3, 3)
@show typeof(M)

In [ ]:
N = ones(Int8, 2,2,2)
@show typeof(N)

In [ ]:
@show size(N)

In [ ]:
@show typeof(size(N))

In [ ]:
O = cat(N, ones(Int8, 2, 2), dims=3)
@show size(O)

In [ ]:
M2 = vcat(M, M)
@show size(M2)

In [ ]:
M2 = hcat(M, M)
@show size(M2)

We can also apply the same functions over multiple elements in a collection:

In [ ]:
x = [rand(2, 3), rand(3, 4), rand(5, 10)] .* [rand(3, 6), rand(4, 2), rand(10, 1)]

We'll create our own "abstract" type to represent the generic idea of a Hamiltonian. We can define generic methods on this type which its child types will inherit.

In [ ]:
abstract type Hamiltonian{N} end

In [ ]:
# let's extend some methods from the LinearAlgebra standard library
using LinearAlgebra
LinearAlgebra.ishermitian(H::Hamiltonian) = true
Base.ndims(H::Hamiltonian{N}) where {N} = N

Now let's build a **mutable** struct to represent the Hamiltonians of a specific model.

In [ ]:
struct TransverseFieldIsing{N} <: Hamiltonian{N}
    dims::NTuple{N, Int}
    h::Array{Float64, N}
    J::Array{Float64, N}
    row_ixs::Vector{Int}
    col_ixs::Vector{Int}
    nz_vals::Vector{ComplexF64}
    # inner c-tor
    function TransverseFieldIsing{N}(dims, h, J, row_ixs, col_ixs, nz_vals) where {N}
        new(dims, h, J, row_ixs, col_ixs, nz_vals)
    end
end

In [ ]:
# outer c-tor
function TransverseFieldIsing(dims, h, J)
    N = length(dims) # number of dimensions in lattice
    ndims(h) != N && throw(ArgumentError("h must have the same number of dimensions as dims!"))
    ndims(J) != N && throw(ArgumentError("J must have the same number of dimensions as dims!"))
    # create indices representing (hypercubic) lattice
    cis = CartesianIndices(dims)
    for (ii, ci) in enumerate(cis)
        # fill this in later...?
    end
end

In [ ]:
# outer c-tor -- let's specialize this a bit!
function TransverseFieldIsing(dims::Tuple{Int}, h::Vector{Float64}, J::Vector{Float64})
    N_sites = dims[1]
    length(h) == 1 || length(h) == N_sites || throw(ArgumentError("h must be either length 1 or length $N_sites"))
    length(J) == 1 || length(J) == N_sites || throw(ArgumentError("J must be either length 1 or length $N_sites"))
    if length(h) == 1
        h = fill(h, N_sites)
    end
    if length(J) == 1
        J = fill(J, N_sites)
    end
    row_ixs = Int[]
    col_ixs = Int[]
    nz_vals = ComplexF64[]
    for input_ind in 0:2^N_sites-1
        input_bits = Vector{Bool}(digits(input_ind, base=2, pad=N_sites))
        # h-term
        @inbounds for ii in 1:N_sites
            output_bits = copy(input_bits)
            output_bits[ii] ⊻= true
            output_ind = sum([output_bits[jj]<<(jj-1) for jj in 1:N_sites])
            push!(row_ixs, input_ind+1)
            push!(col_ixs, output_ind+1)
            push!(nz_vals, h[ii])
        end
        # J-term
        push!(row_ixs, input_ind+1)
        push!(col_ixs, input_ind+1)
        nz_val = 0.0
        @inbounds for site in 1:N_sites-1
            next_site   = site + 1
            flip = (input_bits[site] ^ input_bits[next_site]) == 1
            bond_J = flip ? -J[site] : J[site]
            nz_val += bond_J
        end
        push!(nz_vals, nz_val)
    end
    return TransverseFieldIsing{1}(dims, h, J, row_ixs, col_ixs, nz_vals)
end

In [ ]:
tf = TransverseFieldIsing((2,), [1.0, 1.0], [2.0, 2.0]);

Now let's extend some methods that Julia itself provides to make better use of our new type. At first we will be doing exact diagonalization, so it makes sense to extend some methods from Julia for sparse matrices.

In [ ]:
using SparseArrays
Base.convert(SparseMatrixCSC, tf::TransverseFieldIsing) = sparse(tf.row_ixs, tf.col_ixs, tf.nz_vals)

In [ ]:
convert(SparseMatrixCSC, tf)

In [ ]:
ishermitian(tf)

Unfortunately, Julia doesn't provide a sparse eigensolver by default. For now we can get around this by implementing an ugly hack: converting to a dense matrix. Of course, this won't work for large systems.

In [ ]:
function LinearAlgebra.eigen(tf::TransverseFieldIsing)
    sp_mat = convert(SparseMatrixCSC, tf)
    return eigen(Hermitian(Matrix(sp_mat)), 1:1)
end

In [ ]:
eigen(tf)

Converting to a dense matrix like this isn't very efficient -- can we use a package to do better?

Yes. [Arpack.jl](https://github.com/JuliaLinearAlgebra/Arpack.jl) wraps the ARPACK sparse eigensolving/SVD library and we can use it to perform the diagonalization. To use it, we'll first engage Julia's package manager (helpfully called `Pkg`) to install the package. Note that becaus ARPACK itself is written in FORTRAN, you might get install errors if your system doesn't have a `gfortran` in its `PATH`. 

In [ ]:
using Pkg
Pkg.add("Arpack")

In [ ]:
using Arpack

In [ ]:
function LinearAlgebra.eigen(tf::TransverseFieldIsing)
    sp_mat = convert(SparseMatrixCSC, tf)
    λ, ϕ = eigs(Hermitian(sp_mat), nev=1, which=:SR)
    return λ, ϕ
end

In [ ]:
eigen(tf)

We should also write some basic unit tests for our code to try to catch bugs. Julia provides testing functionality to ensure that a function throws an expected error, or that the result is (approximately) equal to some expected value. We can also `@test_broken` to allow a test to pass which shouldn't and mark it as something we should return to in the future.

In [ ]:
using Test
@test_throws ArgumentError TransverseFieldIsing((2,), [1.0, 1.0, 1.0], [2.0, 2.0])
@test_throws ArgumentError TransverseFieldIsing((2,), [1.0, 1.0], [2.0, 2.0, 2.0])

In [ ]:
tf = TransverseFieldIsing((2,), [0.0, 0.0], [2.0, 2.0])
@test eigen(tf)[1] ≈ [-2.0] atol=1e012